# Úkol č. 1 - předzpracování dat a binární klasifikace

* Termíny jsou uvedeny na [courses.fit.cvut.cz](https://courses.fit.cvut.cz/BI-ML1/homeworks/index.html).
* Pokud odevzdáte úkol po prvním termínu ale před nejzašším termínem, budete penalizování -12 body, pozdější odevzdání je bez bodu.
* V rámci tohoto úkolu se musíte vypořádat s klasifikační úlohou s příznaky různých typů.
* Před tím, než na nich postavíte predikční model, je třeba je nějakým způsobem převést do číselné reprezentace.
    
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

Využívejte buňky typu `Markdown` k vysvětlování Vašeho postupu. Za nepřehlednost budeme strhávat body.

## Zdroj dat

Budeme se zabývat predikcí přežití pasažérů Titaniku.
K dispozici máte trénovací data v souboru `data.csv` a data na vyhodnocení v souboru `evaluation.csv`.

#### Seznam příznaků:
* survived - zda pasažér přežil, 0 = Ne, 1 = Ano, **vysvětlovaná proměnná**, kterou chcete predikovat
* pclass - Třída lodního lístku, 1 = první, 2 = druhá, 3 = třetí
* name - jméno
* sex - pohlaví
* age - věk v letech
* sibsp	- počet sourozenců / manželů, manželek na palubě
* parch - počet rodičů / dětí na palubě
* ticket - číslo lodního lístku
* fare - cena lodního lístku
* cabin	- číslo kajuty
* embarked	- místo nalodění, C = Cherbourg, Q = Queenstown, S = Southampton
* home.dest - Bydliště/Cíl

## Pokyny k vypracování

**Body zadání**, za jejichž (poctivé) vypracování získáte **25 bodů**: 
  * V notebooku načtěte data ze souboru `data.csv`. Vhodným způsobem si je rozdělte na podmnožiny, které Vám poslouží pro trénování (trénovací), porovnávání modelů (validační) a následnou predikci výkonnosti finálního modelu (testovací).
    
  * Proveďte základní předzpracování dat:
    * Projděte si jednotlivé příznaky a transformujte je do vhodné podoby pro použití ve vybraném klasifikačním modelu.
    * Podle potřeby si můžete vytvářet nové příznaky (na základě existujících), například tedy můžete vytvořit příznak měřící délku jména atp.
    * Některé příznaky můžete také úplně zahodit.
    * Nějakým způsobem se vypořádejte s chybějícími hodnotami. _Pozor na metodické chyby!_
    * Můžete využívat i vizualizace a vše stručně ale náležitě komentujte.

  
  * Na připravená data postupně aplikujte **rozhodovací strom** a **metodu nejbližších sousedů**, přičemž pro každý z těchto modelů:
    * Okomentujte vhodnost daného modelu pro daný typ úlohy.
    * Vyberte si hlavní hyperparametry k ladění a najděte jejich nejlepší hodnoty.
    * Pro model s nejlepšími hodnotami hyperparametrů spočtěte F1 skóre, nakreslete ROC křivku a určete AUC. _Pozor na metodické chyby!_
    * Získané výsledky vždy řádně okomentujte.

        
  * Ze všech zkoušených možností v předchozím kroku vyberte finální model a odhadněte, jakou přesnost můžete očekávat na nových datech, která jste doposud neměli k dispozici. _Pozor na metodické chyby!_
    
  * Nakonec načtěte vyhodnocovací data ze souboru`evaluation.csv`. Pomocí finálního modelu napočítejte predikce pro tyto data (vysvětlovaná proměnná v nich již není). Vytvořte soubor `results.csv`, ve kterém získané predikce uložíte do dvou sloupců: **ID**, **survived**. Tento soubor též odevzdejte (uložte do repozitáře vedle notebooku).

  * Ukázka prvních řádků souboru `results.csv`:
  
```
ID,survived
1000,0
1001,1
...
```

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-ML1/homeworks/index.html.

In [108]:
import math
import pandas as pd
import numpy as np
import re
from typing import Union
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
%matplotlib inline

In [109]:
df = pd.read_csv('data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         1000 non-null   int64  
 1   survived   1000 non-null   int64  
 2   pclass     1000 non-null   int64  
 3   name       1000 non-null   object 
 4   sex        1000 non-null   object 
 5   age        802 non-null    float64
 6   sibsp      1000 non-null   int64  
 7   parch      1000 non-null   int64  
 8   ticket     1000 non-null   object 
 9   fare       999 non-null    float64
 10  cabin      233 non-null    object 
 11  embarked   998 non-null    object 
 12  home.dest  566 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 101.7+ KB


Process columns

* pclass - ordinal value, all non null, no change needed
* name - it doesn't seem useful to predict which name will survive and which not, we'll drop this value. However, a name also contains a title of that person, which
can tell us about many things, like a marital status, education, age, ethnicity... Some titles are in a different language, which have equivalent meaning in English,
so I've translated them
* sex - nominal value, no change needed
* age - missing values, useful metrics, we'll predict missing values
* sibsp - no change needed
* parch - no change needed
* ticket - they're usually unique numbers, we wouldn't get much value out of it. However, there seems to be 2 types of tickets, one is that starts with a number and second one starts with a letter. There may be some value hidden there, we'll split ticket into ones that start with a number and ones that start with a letter.
* fare - one value is missing. I've also seen instances when fare was 0. I won't be changing this value and assuming that it was for free. There are no negative numbers, but if they were, I'm assuming that a passenger got paid.
* cabin - a lot of values are missing, but first letter may be useful, as it's probably referencing a place where they stayed in the ship, extract first letter, if cabin matches '\a\d+' regex
* embarked - ordinal value, some is missing
* home.dest - a lot of values are missing. We have both place where they live and a destination here, so we'll need to split the data into 2. The places seem to be separated by ',', so I'll be respecting that. However sometimes there are 3 or even 4 places separated by ','. The first item will always be home and the last destination, I'll ignore else. Also I'll ignore values without ',', since we don't know if it's home or destination

In [110]:
df = pd.read_csv('data.csv')

df.set_index("ID", inplace=True)

translate_title = {
    "Mlle.": "Miss.",
    "Dona.": "Mrs.",
    "Mme.": "Mrs."
}

def get_title(name: str):
    titles = re.findall(r"[A-Z]\S+\.", name)
    if titles:
        title = titles[0]
        if title in translate_title:
            title = translate_title[title]
        return title
    return np.nan

df["title"] = df["name"].apply(get_title)
df = df.drop("name", axis=1)

# df["cabin_letter"] = df["cabin"].apply(lambda x: x[0].upper() if re.fullmatch(r"[ABCDEFGHI]\d+", str(x)) else np.nan)
df["is_in_cabin"] = df["cabin"].apply(lambda x: "In cabin" if type(x) == str else "Not in cabin")
df = df.drop("cabin", axis=1)

def categorize_ticket(ticket: Union[str, float]):
    if ticket == np.nan:  # protection in case future data have null
        return np.nan
    ticket = ticket.strip()
    if ticket[0].isdigit():
        return "Doesn't start with a letter"
    return "Starts with a letter"

df["ticket_starts_with_letter"] = df["ticket"].apply(categorize_ticket)
df = df.drop("ticket", axis=1)

def categorize_home_dest(home_dest: Union[str, float], is_home: bool):
    if type(home_dest) != str or len(home_dest.split(",")) <= 1:
        return np.nan

    if is_home:
        return home_dest.split(",")[0].replace("  ", " ")
    return home_dest.split(",")[-1].replace("  ", " ")

def aaa(x: str):
    if type(x) != str:
        return np.nan
    return x.split()[-1]

df = df.drop("home.dest", axis=1)

df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   survived                   1000 non-null   int64  
 1   pclass                     1000 non-null   int64  
 2   sex                        1000 non-null   object 
 3   age                        802 non-null    float64
 4   sibsp                      1000 non-null   int64  
 5   parch                      1000 non-null   int64  
 6   fare                       999 non-null    float64
 7   embarked                   998 non-null    object 
 8   title                      1000 non-null   object 
 9   is_in_cabin                1000 non-null   object 
 10  ticket_starts_with_letter  1000 non-null   object 
dtypes: float64(2), int64(4), object(5)
memory usage: 93.8+ KB


As we can see, we're still missing 198 numerical values of age and 1 numerical value in fare. We can demonstrate different methods for filling
these fields with values. We also have 2 missing values in embarked, however since it's a nominal value, it shouldn't be such a problem after
one-hot encoding

We'll use linear regression for age and mean for fare

Now convert all columns into data that's in a friendly format for machine learning.

1. Our only ordinal value is pclass, and it's already encoded in numbers and ordered, so we don't need to change it
2. Convert all nominal values (all objects) with one-hot encoding
3. fare is missing 1 value, replace it with mean

In [111]:
df = pd.get_dummies(df)
df['fare'].fillna(df['fare'].mean(), inplace=True)


Predict age

In [112]:
from sklearn.linear_model import LinearRegression


def predict_age(df_final: pd.DataFrame) -> pd.DataFrame:

    # we don't want to use survived for predicting age, as it's a data that we want to predict in the end
    df_predict_age = df_final.drop("survived", axis=1)

    # split the data into ones we'll train on and ones we'll predict
    # Using copy() to prevent warnings
    df_with_age = df_predict_age[df['age'].notna()].copy()
    # since we'll want to work with ages as whole numbers, we'll get rid of the decimal places for training
    df_with_age["age"] = df_with_age["age"].astype(int)
    df_without_age = df_predict_age[df['age'].isna()].copy()

    # split the data into X and Y
    Xtrain = df_with_age.drop("age", axis=1)
    Ytrain = df_with_age["age"]

    # normalize the data
    scaler = MinMaxScaler()
    Xtrain = scaler.fit_transform(Xtrain)

    # teach the model
    linear_regression = LinearRegression()
    linear_regression.fit(Xtrain, Ytrain)

    # predict missing values
    Xpredict = df_without_age.drop("age", axis=1)
    Xpredict = scaler.fit_transform(Xpredict)
    predicted_age = linear_regression.predict(Xpredict)
    df_without_age["age"] = predicted_age

    df_to_attach = pd.concat([df_with_age, df_without_age], axis=0).sort_index()
    df_final["age"] = df_to_attach["age"]

    # get rid of floats, round the age
    df_final["age"] = df_final["age"].apply(lambda x: int(round(x)))

    return df_final


df = predict_age(df)
df


,survived,pclass,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,...,title_Miss.,title_Mr.,title_Mrs.,title_Ms.,title_Rev.,title_Sir.,is_in_cabin_In cabin,is_in_cabin_Not in cabin,ticket_starts_with_letter_Doesn't start with a letter,ticket_starts_with_letter_Starts with a letter
ID,,,,,,,,,,,,,,,,,,,,,
0,0,3,11,5,2,46.9000,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
1,0,3,28,0,0,7.0500,0,1,0,0,...,0,1,0,0,0,0,0,1,0,1
2,0,3,4,3,2,27.9000,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
3,1,3,36,1,0,15.5000,1,0,0,1,...,0,0,1,0,0,0,0,1,1,0
4,1,3,18,0,0,7.2292,1,0,1,0,...,0,0,1,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,2,52,0,0,13.0000,0,1,0,0,...,0,1,0,0,0,0,0,1,1,0
996,0,2,29,0,0,10.5000,0,1,0,0,...,0,1,0,0,0,0,0,1,0,1
997,0,1,56,0,0,26.5500,0,1,0,0,...,0,1,0,0,0,0,0,1,1,0
